In [1]:
# import dependencies 
import numpy as np 
import sys

Using TensorFlow backend.


In [2]:
# load data 
file = open("frankenstein.txt").read()

In [3]:
# tokenize the words from the test file 
# standardizatoin 
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

def tokenize_words(input):
    # making the input word a lower case 
    input = input.lower()
    
    # making tokens using tokenizer 
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)
    
    # filtering the stop words from the tokens 
    filtered = filter(lambda token : token not in stopwords.words('english'), tokens)
    return "".join(filtered)

processed_inputs = tokenize_words(file)

In [4]:
# sorting the char and converting char to num 
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c,i) for i, c in enumerate(chars))

In [5]:
# determining the length of the variabes
input_len = len(processed_inputs)
vocab_len = len(chars)
print('total number of characters : ', input_len)
print('total number of vocabs : ', vocab_len)

total number of characters :  232972
total number of vocabs :  37


In [8]:
# generating out dataset

seq_length = 100
X_data = []
y_data = []

In [9]:
# creating input sequence and output sequence 

for i in range(0, input_len - seq_len):
    in_seq = processed_inputs[i:i + seq_length]
    out_seq = processed_inputs[i + seq_length]
    # converting the sequences in numbers and adding it to our dataset
    X_data.append([char_to_num[char] for char in in_seq])
    y_data.append([char_to_num[out_seq]])

# note : try printin an inividual in_seq and out_seq for better understanding 

In [10]:
# number of patters we have 
n_patterns = len(X_data)
print('total patterns : ', n_patterns)

total patterns :  232872


In [11]:
# converting the input sequence to numpy array 
X = np.reshape(X_data, (n_patterns, seq_length, 1))
X = X/float(vocab_len)

In [12]:
# one_hot encoding 
from keras.utils import np_utils 
y = np_utils.to_categorical(y_data)

In [20]:
# creating a sequencial model 
from keras import Sequential
from keras.layers import Dense, Dropout, LSTM
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [21]:
# compiling the model 
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [23]:
# saving weights
from keras.callbacks import ModelCheckpoint
filepath = "model_weights_saved.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]

In [25]:
# fit the model and let it train 
model.fit(X, y, epochs=4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
  1536/232872 [..............................] - ETA: 1:03:06 - loss: 3.0072

KeyboardInterrupt: 

In [26]:
# recompiling the model with the saved weights 
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

OSError: Unable to open file (unable to open file: name = 'model_weights_saved.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
# output of the model back into the characters 
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
# generating sequence of characters using the random seed

start = numpy.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

In [ ]:
# generating the text 

for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(vocab_len)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = num_to_char[index]
    seq_in = [num_to_char[value] for value in pattern]

    sys.stdout.write(result)

    pattern.append(index)
    pattern = pattern[1:len(pattern)]